In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, confusion_matrix

import sys
print (sys.version)

# need to work with excel formatted dates and re:
import datetime
import re
from functools import reduce

%matplotlib inline

In [ ]:
data=r'C:\Users\joogl\Assessments\Fair Assessment\Fair Data\train_data.csv'
df = pd.read_csv(data, nrows=200000, parse_dates=['earliest_cr_line', 'apply_date'])
df.head(10)

In [ ]:
# Loan Characteristics (Text)
print('LOAN PURPOSE:')
print(df['purpose'].unique())
print("--------------------------")
print('LOAN STATUS:')
print(df['loan_status'].unique())
print("--------------------------")
print('LOAN TITLE:')
print(df['title'].unique())

In [ ]:
# Borrower Characteristics (Text)
print('BORROWER STATE:')
print(df['addr_state'].unique())
print(df.groupby(['addr_state']).size())
print("--------------------------")
print('BORROWER JOB TITLE:')
print(df['emp_title'].unique())
print("--------------------------")
print('BORROWER HOME TYPE:')
print(df['home_ownership'].unique())


I. Explore Continuous Data


## Clean Continuous Data

In [ ]:
df_dti = df['dti']
print('dti --> (avg monthly debt payments/annual income): min: {}, max: {}, mean: {}, sd: {}'.format(
    round(df_dti.min(), 2),
    round(df_dti.max(), 2),
    round(df_dti.mean(), 2),
    round(df_dti.std(), 2)
))

In [ ]:
look_up = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May',
            '06': 'Jun', '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
inv_map = {v: k for k, v in look_up.items()}


In [ ]:
# Clean earliest_cr_line

new_credit_date = pd.DataFrame([], columns=['credit_date'])
print(new_credit_date)

fd_date = pd.to_datetime(df['earliest_cr_line'], errors='coerce')
fd_date = fd_date.map(lambda x: x - pd.DateOffset(years=100) if x.year>2018 else x)

for n, element in enumerate(fd_date):
    #print("---------------")
    #print(n)
        
    if str(element) == 'NaT':
        if (str(df['earliest_cr_line'][n]))=='nan':
            fd_date = np.nan
        elif str(df['earliest_cr_line'][n]).split("-")[1] == '00':
            fd_date = datetime.datetime(2000, int(inv_map[df['earliest_cr_line'][n].split("-")[0]]), 1).date()
        else:
            fd_date = datetime.datetime(2017, datetime.datetime.strptime(df['earliest_cr_line'][n], '%d-%b').month, \
                                    datetime.datetime.strptime(df['earliest_cr_line'][n], '%d-%b').day).date()
        #print(fd_date)
        new_credit_date = new_credit_date.append({'credit_date': fd_date}, ignore_index=True)
    else:
        #print(element)
        new_credit_date = new_credit_date.append({'credit_date': element.date()}, ignore_index=True)


In [ ]:
# Clean apply_date. All years are 2017.

apply_dates = pd.to_datetime(df.apply_date, format='%d-%b').apply(lambda x: x + pd.DateOffset(years=117))
print(apply_dates[0:5])

In [ ]:
# Clean emp_length (need to get integer)
emp_length_int = df['emp_length'].str.replace(r'\D+', '')
print(emp_length_int[0:5])

In [ ]:
# Ignore emp_title (probably highly correlated with income even when cleaned)
# Ignore mths_since_last_major_derog (too sparse)
# Ignore Loan Title (too many categories and probably highly correlated with loan purpose)
# Ignore verification_status_joint (too sparse)

In [ ]:
# Transform new_credit_date to days since earliest credit line (use timedelta)

df_crl = new_credit_date['credit_date']

present_date = pd.DataFrame(index=range(len(df_crl)), columns=['tdate'])
present_date['tdate'] = pd.Timestamp('2018-03-01')
t_date = pd.to_datetime(present_date.tdate, format='%Y-%m-%d')
c_date = pd.to_datetime(df_crl, format='%Y-%m-%d')

#print(t_date[0:5])
#print(c_date[0:5])

credit_days = t_date - c_date 
#print(credit_days)

credit_days_df = pd.DataFrame(credit_days, columns=['days'])
#print(credit_days_df[0:5])
credit_days_df = credit_days_df['days'].astype(datetime.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
print(credit_days_df[0:5])

In [ ]:
# Transform new_credit_date to days since earliest credit line (use timedelta)

apply_days = t_date - apply_dates
#print(apply_days)

apply_days_df = pd.DataFrame(apply_days, columns=['days'])
#print(apply_days_df[0:5])
apply_days_df = apply_days_df['days'].astype(datetime.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
print(apply_days_df[0:5])

In [ ]:
print('credit days --> (number of days since first credit line opened): min: {}, max: {}, mean: {}, sd: {}'.format(
    round(credit_days_df.min(), 2),
    round(credit_days_df.max(), 2),
    round(credit_days_df.mean(), 2),
    round(credit_days_df.std(), 2)
))

In [ ]:
print('application days --> (number of days since application opened): min: {}, max: {}, mean: {}, sd: {}'.format(
    round(apply_days_df.min(), 2),
    round(apply_days_df.max(), 2),
    round(apply_days_df.mean(), 2),
    round(apply_days_df.std(), 2)
))

In [ ]:
rm_credit = df['revol_bal']
print('revol_bal --> (Remaining Credit Line): min: {}, max: {}, mean: {}, sd: {}'.format(
    round(rm_credit.min(), 2),
    round(rm_credit.max(), 2),
    round(rm_credit.mean(), 2),
    round(rm_credit.std(), 2)
))

In [ ]:
ann_income = df['annual_inc']
print('annual_inc --> (Annual Income): min: {}, max: {}, mean: {}, sd: {}'.format(
    round(ann_income.min(), 2),
    round(ann_income.max(), 2),
    round(ann_income.mean(), 2),
    round(ann_income.std(), 2)
))

## One-Hot Encoding on String Variables

In [ ]:
def hot_encoding(pd_column):
    # Purpose: applies 1 hot encoding to categrical variables
    # Output: pd dataframe. Columns use dataframe input column name as prefix
    prefix = pd_column.name
    
    dummy = OneHotEncoder()
    dummyC = LabelEncoder()
    df_cat2 = np.zeros((pd_column.shape[0], 1))
    y = dummyC.fit_transform(pd_column.reshape(-1, 1))
    y = dummy.fit_transform(y.reshape(-1, 1)).toarray()
    y = pd.DataFrame(y[:, 1:])
    df_cat2 = np.hstack((df_cat2, y))
    
    df_stuff = pd.DataFrame(df_cat2)
    df_stuff.columns = [str(prefix)+str("_")+str(n) for n in list(df_stuff.columns)]
    
    return pd.DataFrame(df_stuff)

In [ ]:
# Txt - Geography

geog = hot_encoding(df['addr_state'])
print(geog)

In [ ]:
# Txt - Home Type
home_type = hot_encoding(df['home_ownership'])
print(home_type)

In [ ]:
# Txt - Loan Status
# loan_status = hot_encoding(df['loan_status'])
# avoid using function since we want to specifically drop the default category and use the rest of the data as covariates

prefix = df['loan_status'].name

dummy = OneHotEncoder()
dummyC = LabelEncoder()
df_cat2 = np.zeros((df['loan_status'].shape[0], 1))
y = dummyC.fit_transform(df['loan_status'].reshape(-1, 1))
y = dummy.fit_transform(y.reshape(-1, 1)).toarray()

df_cat2 = np.hstack((df_cat2, y))

df_stuff = pd.DataFrame(df_cat2)
df_stuff.columns = [str(prefix)+str("_")+str(n) for n in list(df_stuff.columns)]
# drop default variable
# test: df['loan_status'].unique()
# test: df['loan_status'][702]
# test: loan_status['loan_status_3'][2296]
 
loan_status = pd.DataFrame(df_stuff)
loan_status.drop(['loan_status_3'], axis=1, inplace=True)    # loan status = default
loan_status.drop(['loan_status_1'], axis=1, inplace=True)    # loan status = charged-off
loan_status.drop(['loan_status_10'], axis=1, inplace=True)   # loan status = 30+ days late
print(loan_status)

In [ ]:
# Txt - loan Purpose
loan_purpose = hot_encoding(df['purpose'])
print(loan_purpose)

In [ ]:
# Bool - Verfied Income
verified_income = hot_encoding(df['verification_status'])
print(verified_income)

In [ ]:
## Create a merged dataframe from the following vectors:

# verified_income
# loan_purpose
# loan_status
# home_type
# geog
#
# ann_income
# rm_credit
# apply_days_df
# credit_days_df
# df_dti
# emp_length_int

"""
plt.hist(df_dti)
plt.hist(emp_length_int)
plt.hist(credit_days_df.dropna(axis=0, how='any'))
plt.hist(rm_credit)
plt.hist(ann_income.dropna(axis=0, how='any'))

print(len(ann_income)) #
print(len(rm_credit)) #
print(len(apply_days_df)) #
print(len(credit_days_df)) #
print(len(df_dti)) #
print(len(emp_length_int)) #
print(len(verified_income))
print(len(loan_purpose))
print(len(loan_status))
print(len(home_type))
print(len(geog))
"""

In [ ]:
ann_income_pd = ann_income.to_frame('ann_income')
rm_credit_pd = rm_credit.to_frame('rm_credit')
apply_days_pd = apply_days_df.to_frame('apply_days')
credit_days_pd = credit_days_df.to_frame('credit_days')
dti_pd = df_dti.to_frame('dti')
emp_length_pd = emp_length_int.to_frame('emp_length')

In [ ]:
df_default = df.loan_status.apply(lambda x: 1 if x in {'Default', 'Late (31-120 days)', 'Charged Off'} else 0)
df_default = df_default.to_frame('default')

In [ ]:
dfs = [df_default, ann_income_pd, rm_credit_pd, apply_days_pd, credit_days_pd, dti_pd, emp_length_pd, \
      verified_income, loan_purpose, loan_status, home_type, geog]
dfs = [df.reset_index() for df in dfs]

df_final = reduce(lambda left,right: pd.merge(left,right,on='index'), dfs)

In [ ]:
# drop any rows with NA before analysis (if regressions fail)
#print(len(df_final))
df_final = df_final.dropna(axis=0, how='any')
#print(len(df_final))

## Begin Modeling the Training Set

In [ ]:
X = df_final[[x for x in df_final.columns if x != 'default' and x != 'index']]
y = df_final[['default']].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [ ]:
#print(len(X_test))
X_test = X_test.dropna(axis=0, how='any')
#print(len(X_test))
#print(len(y_test))
#print(y_test)

#print(len(X_train))
X_train = X_train.dropna(axis=0, how='any')
#print(len(X_train))
#print(len(y_train))
#print(y_train)

In [ ]:
# scale features to have same variance
sc = StandardScaler()
X_train_cont = sc.fit_transform(X_train)
X_test_cont = sc.transform(X_test)

In [ ]:
def create_model(model, X, y):
    if model == 'lr':
        myModel = LogisticRegression(random_state=42, class_weight='balanced', n_jobs=-1)
    elif model == 'svm':
        myModel = SVC(kernel='linear', random_state=42, class_weight='balanced', cache_size=2048)
    elif model == 'rf':
        myModel = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
    else:
        raise error('cannot fit that model')
    
    myModel.fit(X, y)
    
    if model == 'lr':
        y_pred_probs = myModel.predict_proba(X)
        y_pred = myModel.predict(X)
    else:
        y_pred = myModel.predict(X)
        in_acc = accuracy_score(y_pred, y)
    print(y_pred)
    print(y)
    in_RMSE = np.sqrt(mean_squared_error(y_pred, y))
    in_acc = accuracy_score(y_pred, y)
    tn, fp, fn, tp = confusion_matrix(y_pred, y).ravel()
    print('Accuracy: {}, Precision: {}, Recall: {}'.format(round(in_acc, 2), tp / (tp + fp), tp / (tp + fn)))
    print(pd.DataFrame(confusion_matrix(y_pred, y)))
    
    return y_pred, myModel, in_RMSE

In [ ]:
pred1, model1, in_RMSE1 = create_model('lr', X_train, y_train[:,0])
# pred2 = create_model('svm', X_train, y_train[:,0])
pred3, model3, in_RMSE3 = create_model('rf', X_train, y_train[:,0])

print("Done.")

In [ ]:
lr_pred = pd.Series(pred1 * 0.67)
# svm_pred = pd.Series(pred2 * 0.79)
rf_pred = pd.Series(pred3 * 0.96)

## Weighted Precision (using confusion matrix)

In [ ]:
final_pred = pd.DataFrame([lr_pred, rf_pred]).sum()
final_pred = final_pred.apply(lambda x: 1 if x > 1 else 0)
in_acc = accuracy_score(final_pred, y_train[:,0])
tn, fp, fn, tp = confusion_matrix(final_pred, y_train[:,0]).ravel()
print('Accuracy: {}, Precision: {}, Recall: {}'.format(round(in_acc, 2), tp / (tp + fp), tp / (tp + fn)))
confusion_matrix(final_pred, y_train[:,0])

## Begin Modeling the Test Set (using Logistic Regression Model)

In [ ]:
y_pred = model3.predict(X_test)

out_acc = accuracy_score(y_pred, y_test[:,0])
out_RMSE = np.sqrt(mean_squared_error(y_pred, y_test[:,0]))
                   
tn, fp, fn, tp = confusion_matrix(y_pred, y_test[:,0]).ravel()
print('In-sample accuracy: {}, Precision: {}, Recall: {}'.format(round(out_acc, 2), tp / (tp + fp), tp / (tp + fn)))
confusion_matrix(y_pred, y_test[:,0])

In [ ]:
# get in and out of sample accuracy
# pred1
# in_RMSE3

rSquared = model3.score(X_test, y_test[:,0])
print('In-sample RMSE: {}\nOut-of-sample RMSE: {}\nRsquared: {}'.format(in_RMSE3, out_RMSE, rSquared))

In [ ]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(model1.coef_))], axis = 1)

In [ ]:
print(coefficients)
print("End.")